In [1]:
import sys
sys.path.append('../')
from mylibs import *

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
from data.data import Data
handler = Data()
data = handler.load('../artifacts_5s_128hz.pkl')
train_split, val_split, test_split = handler.split(data)

In [3]:
train_features, train_labels = handler.prepare_data(train_split, balance=True)

In [4]:
train_features = train_features[:2, :, :]
train_labels = train_labels[:2, :, :]

In [5]:
from architectures.BidirectionalLSTM import BidirectionalLSTM

In [6]:
model = BidirectionalLSTM(num_hidden_units=16, num_bidirectional_layers=1)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [7]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy())

In [8]:
model.fit(train_features, train_labels, epochs=1)

1/1 [==============================] - 6s 6s/step - loss: 0.6978


In [9]:
from architectures.MultiLayerLSTM import MultiLayerLSTM

In [10]:
model2 = MultiLayerLSTM(num_hidden_units=16, num_lstm_layers=1)

In [11]:
model2.compile(loss=tf.keras.losses.BinaryCrossentropy())

In [12]:
model2.fit(train_features, train_labels, epochs=1)

1/1 [==============================] - 6s 6s/step - loss: 0.6716


In [21]:
model = keras.models.Sequential()
model.add(layers.TimeDistributed(layers.Conv1D(filters=32, kernel_size=1, padding='same', activation='relu', input_shape=(640, 19))))
model.add(layers.Conv1D(filters=32, kernel_size=1, padding='same', activation='relu'))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(2, activation='softmax'))

In [23]:
model.build()
model.summary()

ValueError: You must provide an `input_shape` argument.

In [25]:
inputs = tf.keras.Input(shape=(640, 19))
conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3))
outputs = tf.keras.layers.TimeDistributed(conv_2d_layer)(inputs)
outputs = keras.layers.LSTM(32, return_sequences=True)(outputs)
output = keras.layers.Dense(2)
outputs.shape



ValueError: Exception encountered when calling layer "time_distributed_2" (type TimeDistributed).

Input 0 of layer "conv2d_1" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 19)

Call arguments received by layer "time_distributed_2" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 640, 19), dtype=float32)
  • training=None
  • mask=None

In [40]:
cnn = keras.models.Sequential()
cnn.add(layers.Conv1D(1, 3, activation='relu', padding='same', input_shape=(640,19)))
cnn.add(layers.MaxPooling1D(pool_size=(2)))
cnn.add(layers.Flatten())


lstm = keras.models.Sequential()
lstm.add(layers.TimeDistributed(cnn, input_shape=(640,19)))
lstm.add(layers.LSTM(32, return_sequences=True))
lstm.add(layers.Dense(2))


inputs = tf.keras.Input(shape=(None,640,19))

lstm(inputs).shape

ValueError: Exception encountered when calling layer "time_distributed_10" (type TimeDistributed).

Input 0 of layer "sequential_26" is incompatible with the layer: expected shape=(None, 640, 19), found shape=(None, 19)

Call arguments received by layer "time_distributed_10" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 640, 19), dtype=float32)
  • training=None
  • mask=None